# Principal Component Analysis

Think of the predictors in your dataset as dimensions in what we can usefully call "feature space". If we're predicting house prices, then we might have a 'square feet' dimension or a 'number of bathrooms' dimension, etc. Then each record (of a house or a house sale, say) would be represented as a point (or vector) in this feature space. Some would score higher on the 'latitude' dimension or lower on the 'number of bedrooms' dimension, or whatever.

One difficulty is that, despite our working nomenclature, these things aren't really *dimensions* in the truest sense, since they're not independent of each other. When we talk about the x-, y-, and z-dimensions of Euclidean 3-space, for example, one important feature is that values of x have no bearing (per se) on values of y or of z. I can move three units along the x-dimension without changing my y- or z-position.

But the analogous thing is generally not true for datasets. When I increase my position along the 'number of bedrooms' dimension (or, better, *direction*), I also tend to increase my position along, say, the 'square feet' direction as well.

This is problematic for a couple reasons: One is that my model could be in effect "double-counting" certain features of my signal, which can lead to overfit models. And if my goal is inference or explanation, then I'm going to have a very hard time distinguishing between the idea that the number of bedrooms is what's *really* predictive of housing prices and the idea that the number of square feet is what's really so predictive.

The idea behind Principal Component Analysis (PCA) is to transform our dataset into something more useful for building models. What we want to do is to build new dimensions (predictors) out of the dimensions we are given in such a way that:

(1) each dimension we draw captures as much of the remaining variance among our predictors as possible; and <br/>
(2) each dimension we draw is orthogonal to the ones we've already drawn.

## Motivation

Think back to multiple linear regression for a moment.

The fundamental idea is that I can get a better prediction for my dependent variable by considering a *linear combination of my predictors* than I can get by considering any one predictor by itself.

$\rightarrow$ **PCA insight**: If the combinations of predictors work better than the predictors themselves, then let's just treat the combinations as our primary dimensions!

But one problem with having lots of predictors is that it raises the chance that some will be nearly *collinear*.

$\rightarrow$ **PCA insight**: Since we're reconstructing our dimensions anyway, let's make sure that the dimensions we construct are mutually orthogonal! <br/>
$\rightarrow$ **PCA insight**: Moreover, since we'll be capturing much of the variance among our predictors in the first few dimensions we construct, we'll be able in effect to *reduce  the dimensionality* of our problem. Thus PCA is a fundamental tool in *dimensionality reduction*.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [ ]:
cars = pd.read_csv('cars.csv')

In [ ]:
cars.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cars.drop('mpg', axis=1),
                                                    cars['mpg'],
                                                   random_state=20)

In [ ]:
X_train[' cubicinches'].replace(' ', np.nan, inplace=True)
X_train[' cubicinches'] = X_train[' cubicinches'].map(float)
X_train[' cubicinches'].fillna(X_train[' cubicinches'].mean(skipna=True), inplace=True)

In [ ]:
X_train[' weightlbs'].replace(' ', np.nan, inplace=True)
X_train[' weightlbs'] = X_train[' weightlbs'].map(float)
X_train[' weightlbs'].fillna(X_train[' weightlbs'].mean(), inplace=True)

In [ ]:
X_train[' cylinders'] = X_train[' cylinders'].map(float)
X_train[' hp'] = X_train[' hp'].map(float)
X_train[' time-to-60'] = X_train[' time-to-60'].map(float)
X_train[' year'] = X_train[' year'].map(float)

In [ ]:
# Scaling

ss = StandardScaler()

In [ ]:
# Scale-transforming

X_tr_sc = ss.fit_transform(X_train.drop(' brand', axis=1))

In [ ]:
# Let's construct a linear regression

lr = LinearRegression().fit(X_tr_sc, y_train)

# Score on train
lr.score(X_tr_sc, y_train)

In [ ]:
def clean(df):
    new = df.copy()
    for col in [' cubicinches', ' weightlbs']:
        new[col] = new[col].replace(' ', np.nan).map(float).replace(np.nan, new[col].mean())
    return new

In [ ]:
def to_float(df):
    new = df.copy()
    for col in [' cylinders', ' hp', ' time-to-60', ' year']:
        new[col] = new[col].map(float)
    return new

In [ ]:
def drop(df):
    return df.drop(' brand', axis=1)

In [ ]:
def scale(df, ss):
    return ss.transform(df)

In [ ]:
test_cleaned = clean(X_test)
test_floated = to_float(test_cleaned)
test_dropped = drop(test_floated)
test_scaled = scale(test_dropped, ss)

In [ ]:
# Score on test

lr.score(test_scaled, y_test)

In [ ]:
# Get the coefficients of the best-fit hyperplane

lr.coef_

Thus, our best-fit hyperplane is given by:

$-1.409\times cyl + 0.681\times in^3 - 0.480\times hp - 4.658\times lbs. -  0.176\times time_{60} + 2.427\times yr$

## Eigenvalues and Eigenvectors

The key idea is to diagonalize (i.e. find the eigendecomposition of) the covariance matrix. The decomposition will produce a set of orthogonal vectors that explain as much of the remaining variance as possible. These are our [principal components](https://math.stackexchange.com/questions/23596/why-is-the-eigenvector-of-a-covariance-matrix-equal-to-a-principal-component).

Let's say a word about eigenvalues and eigenvectors. It turns out that eigenvalues and -vectors have a dizzying number of applications. But the basic idea is that, if we can split a bunch of vectors (i.e. a matrix) into a set of mutually orthogonal vectors, then we can isolate the force of the bunch into discrete bits, each of which by itself acts like a simple linear transformation.

That's why the definition of an eigenvector is as it is: $\vec{x}$ is an eigenvector of the matrix $A$ if $A\vec{x} = \lambda\vec{x}$, for some scalar $\lambda$. That is, the vector is oriented in just such a direction that multiplying the matrix by it serves only to lengthen or shorten it.

Suppose we have the matrix
$A =
\begin{bmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22} \\
\end{bmatrix}
$.

Let's calculate the eigendecomposition of this matrix.

In order to do this, we set $(A - \lambda I)\vec{x} = 0$. One trivial solution is $\vec{x} = \vec{0}$, but if there are more interesting solutions, then it must be that $|A - \lambda I| = 0$, which is to say that some column vector in $A - \lambda I$ must be expressible as a linear combination of the other columns. (Otherwise, there would be no way to "undo" the multiplicative effect of a column vector on $\vec{x}$!) For more on this point, see [this page](http://www2.math.uconn.edu/~troby/math2210f16/LT/sec1_7.pdf).

So we have:

$\begin{vmatrix}
a_{11} - \lambda & a_{12} \\
a_{21} & a_{22} - \lambda
\end{vmatrix} = 0$

$(a_{11} - \lambda)(a_{22} - \lambda) - a_{12}a_{21} = 0$

$\lambda^2 - (a_{11} + a_{22})\lambda + a_{11}a_{22} - a_{12}a_{21}$

$\lambda = \frac{a_{11} + a_{22}\pm\sqrt{(a_{11} + a_{22})^2 + 4(a_{12}a_{21} - a_{11}a_{22})}}{2}$

Suppose e.g. we had

$A = \begin{bmatrix}
5 & 3 \\
3 & 5
\end{bmatrix}$.

We can use the equation we just derived to solve for the eigenvalues of this matrix. Then we can plug *those* into our eigenvector definition to solve for the eigenvectors:

So:

### Eigenvalues

$\lambda = \frac{5+5\pm\sqrt{(5+5)^2+4(3\times 3 - 5\times 5)}}{2} = 5\pm\frac{\sqrt{36}}{2} = 2, 8$.

### Eigenvectors

Now we can plug those in. If we plug in $\lambda = 8$, then we get:

$\begin{bmatrix}
5-8 & 3 \\
3 & 5-8
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}
=
\begin{bmatrix}
-3 & 3 \\
3 & -3
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix} = 0.$

So:

$-3x_1 + 3x_2 = 0$ (or $3x_1 - 3x_2 = 0$)

$x_1 = x_2$.

It is standard to scale eigenvectors to a magnitude of 1, and so we would write this eigenvector as
$\begin{bmatrix}
\frac{\sqrt{2}}{2} \\
\frac{\sqrt{2}}{2}
\end{bmatrix}$.

If we plug in $\lambda = 2$, we find a second eigenvector equal to
$\begin{bmatrix}
-\frac{\sqrt{2}}{2} \\
\frac{\sqrt{2}}{2}
\end{bmatrix}$. (I'll leave this as an exercise.)

### In Code

In [ ]:
# We can use np.linalg.eig()

A = np.array([[5, 3], [3, 5]])
np.linalg.eig(A)

In [ ]:
# np.linalg.eig(X) returns a double of NumPy arrays, the first containing
# the eigenvalues of X and the second containing the eigenvectors of X.

v, q = np.linalg.eig(A)

In [ ]:
v

In [ ]:
# np.diag()

np.diag(v)

In [ ]:
# Reconstruct A by multiplication

q.dot(np.diag(v)).dot(q.T)

## PCA by Hand

What follows is indebted to [Sebastian Raschka](http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html#pca-vs-lda).

In [ ]:
# We'll start by producing the covariance matrix for the columns of X_tr_sc.

cov_mat = np.cov(X_tr_sc, rowvar=False)
cov_mat.shape

In [ ]:
cov_mat

In [ ]:
np.linalg.eig(cov_mat)

In [ ]:
# Let's assign the results of eig(cov_mat) to a double of variables.

eigvals, eigvecs = np.linalg.eig(cov_mat)

In [ ]:
# The columns of "eigvecs" are the eigenvectors!

eigvecs

In [ ]:
# The eigenvectors of the covariance matrix are our principal components.
# Let's look at the first three.

pcabh = np.vstack([row[:3].reshape(1, 3) for row in eigvecs])

Now, to transform our data points into the space defined by the principal components, we simply need to compute the dot-product of `X_tr_sc` with those principal components.

Why? Think about what this matrix product looks like:

We take a row of `X_tr_sc` and multiply it by a column of `pcabh`, pairwise. The row of `X_tr_sc` represents the values for the columns in the original space. The column of `pcabh` represents the weights we need on each of the original columns in order to transform a value into principal-component space. And so the product of these two matrices will be each row, transformed into principal-component space!

In [ ]:
X_tr_sc[:5, :]

In [ ]:
X_tr_sc.dot(pcabh)[:5, :]

In [ ]:
# Naturally, sklearn has a shortcut for this!

pca = PCA(n_components=3)                       # Check out how `n_components` works

X_train_new = pca.fit_transform(X_tr_sc)

X_train_new

In [ ]:
# Let's check out the explained variance

pca.explained_variance_

In [ ]:
# The ratio is often more informative

pca.explained_variance_ratio_

In [ ]:
# We can also check out the Principal Components themselves

pca.components_

In [ ]:
# Recall the columns of cars

cars.columns

The results of our PCA are as follows:

**PC1** = 0.454 * cylinders + 0.470 * cubicinches + 0.462 * hp + 0.440 * weightlbs - 0.357 * time-to-60 - 0.196 * year

**PC2** = -0.143 * cylinders - 0.110 * cubicinches - 0.023 * hp - 0.217 * weightlbs - 0.102 * time-to-60 - 0.954 * year

**PC3** = 0.204 * cylinders + 0.153 * cubicinches - 0.129 * hp + 0.361 * weightlbs + 0.860 * time-to-60 - 0.219 * year

It turns out that these loadings are encoded in the eigenvectors of $X^TX$. Notice that:

- the absolute values of the components of PC1 are the first components of the eigenvectors below, <br/>
- the absolute values of the components of PC2 are the second components of the eigenvectors below, <br/>
- etc. <br/>

We'll have more to say about this when we examine the singular value decomposition of matrices later on.

In [ ]:
np.linalg.eig(X_tr_sc.T.dot(X_tr_sc))

## Normality

In [ ]:
# These principal components should be normalized.
# If they are, then the sum of the squares of the
# loadings should be 1. Let's check!

mag0 = 0
for i in range(6):
    mag0 += pca.components_[0][i]**2
mag0

In [ ]:
np.linalg.norm(pca.components_[0])

In [ ]:
np.linalg.norm(pca.components_[1])

In [ ]:
np.linalg.norm(pca.components_[2])

## Orthogonality

In [ ]:
# These principal components should also be
# mutually orthogonal. If they are, then the
# dot product of any two of them should be 0.
# Let's check!

dot_prod01 = 0
for i in range(6):
    dot_prod01 += pca.components_[0][i] * pca.components_[1][i]
dot_prod01

In [ ]:
pca.components_[0].dot(pca.components_[1])

In [ ]:
pca.components_[0].dot(pca.components_[2])

In [ ]:
pca.components_[1].dot(pca.components_[2])

## Visualizations

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

fig, ax = plt.subplots()
ax.plot(X_train_new[:, 0], y_train, 'r.');

In [ ]:
fig, ax = plt.subplots()
ax.plot(X_train_new[:, 1], y_train, 'g.');

In [ ]:
fig, ax = plt.subplots()
ax.plot(X_train_new[:, 2], y_train, 'k.');

Question: Is the first principal component the same line we would get if we constructed an ordinary least-squares regression line?

The answer is NO! Check out this post for an illuminating discussion: https://shankarmsy.github.io/posts/pca-vs-lr.html

## Modeling with New Dimensions

Now that we have optimized our features, we can build a new model with them!

In [ ]:
lr_pca = LinearRegression()
lr_pca.fit(X_train_new, y_train)
lr_pca.score(X_train_new, y_train)

In [ ]:
X_test_new = pca.transform(test_scaled)

In [ ]:
lr_pca.score(X_test_new, y_test)

In [ ]:
lr_pca.coef_

Thus, our best-fit hyperplane is given by:

$-3.00\times PC1 - 1.15\times PC2 -2.49\times PC3$

Of course, since the principal components are just linear combinations of our original predictors, we could re-express this hyperplane in terms of those original predictors!

And if the PCA was worth anything, we should expect the new linear model to be *different from* the first!

Recall that we had:

**PC1** = 0.454 * cylinders + 0.470 * cubicinches + 0.462 * hp + 0.440 * weightlbs - 0.357 * time-to-60 - 0.196 * year

**PC2** = -0.143 * cylinders - 0.110 * cubicinches - 0.023 * hp - 0.217 * weightlbs - 0.102 * time-to-60 - 0.954 * year

**PC3** = 0.204 * cylinders + 0.153 * cubicinches - 0.129 * hp + 0.361 * weightlbs + 0.860 * time-to-60 - 0.220 * year

Therefore, our new PCA-made hyperplane can be expressed as:

$-3.00\times(0.454\times cyl + 0.470\times in^3 + 0.462\times hp + 0.440\times lbs. - 0.357\times time_{60} - 0.196\times yr)$ <br/> $- 1.15\times(-0.143\times cyl - 0.110\times in^3 - 0.023\times hp - 0.217\times lbs. - 0.102\times time_{60} - 0.954\times yr)$ <br/> $- 2.49\times(0.204\times cyl + 0.153\times in^3 - 0.129\times hp + 0.361\times lbs. + 0.860\times time_{60} - 0.219\times yr)$ <br/><br/> $= - 1.706\times cyl - 1.664\times in^3 -1.038\times hp - 1.969\times lbs. - 0.953\times time_{60} + 2.230\times yr$

Our first linear regression model had:

$-1.409\times cyl + 0.681\times in^3 - 0.480\times hp - 4.658\times lbs. -  0.176\times time_{60} + 2.427\times yr$,

which is clearly a different hyperplane.

[Here's](https://www.youtube.com/watch?v=_UVHneBUBW0) a helpful video introduction to PCA if you're itching for more!

## Plotting with Our New Dimensions

In [ ]:
pca.components_

In [ ]:
cars[' cubicinches'].replace(' ', np.nan, inplace=True)
cars[' cubicinches'] = cars[' cubicinches'].map(float)
cars[' cubicinches'].fillna(cars[' cubicinches'].mean(skipna=True), inplace=True)

In [ ]:
cars[' weightlbs'].replace(' ', np.nan, inplace=True)
cars[' weightlbs'] = cars[' weightlbs'].map(float)
cars[' weightlbs'].fillna(cars[' weightlbs'].mean(), inplace=True)

In [ ]:
cars['PC1'] = pca.components_[0][0] * cars[' cylinders'] +\
            pca.components_[0][1] * cars[' cubicinches'] +\
            pca.components_[0][2] * cars[' hp'] +\
            pca.components_[0][3] * cars[' weightlbs'] +\
            pca.components_[0][4] * cars[' time-to-60'] +\
            pca.components_[0][5] * cars[' year']

In [ ]:
cars['PC2'] = pca.components_[1][0] * cars[' cylinders'] +\
            pca.components_[1][1] * cars[' cubicinches'] +\
            pca.components_[1][2] * cars[' hp'] +\
            pca.components_[1][3] * cars[' weightlbs'] +\
            pca.components_[1][4] * cars[' time-to-60'] +\
            pca.components_[1][5] * cars[' year']

In [ ]:
cars['PC3'] = pca.components_[2][0] * cars[' cylinders'] +\
            pca.components_[2][1] * cars[' cubicinches'] +\
            pca.components_[2][2] * cars[' hp'] +\
            pca.components_[2][3] * cars[' weightlbs'] +\
            pca.components_[2][4] * cars[' time-to-60'] +\
            pca.components_[2][5] * cars[' year']

In [ ]:
plt.scatter(cars['PC2'], cars['PC1']);

In [ ]:
plt.scatter(cars['PC3'], cars['PC1']);

In [ ]:
fig, ax = plt.subplots()

ax.scatter(cars['PC3'], cars['PC2']);

In [ ]:
cars.corr()['PC1']